# Imports

In [1]:
from datetime import datetime
import numpy as np
import os
import pandas as pd
from pandas import json_normalize
import requests

import sagemaker
import boto3

c:\Users\sadai\Desktop\AQI-Prediction\.venv\lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[04/22/25 09:35:25] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=211756;file://c:\Users\sadai\Desktop\AQI-Prediction\.venv\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=967522;file://c:\Users\sadai\Desktop\AQI-Prediction\.venv\lib\site-packages\botocore\credentials.py#1352\1352]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\sadai\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
os.getcwd()
os.chdir('..')

In [3]:
AQOPEN_API_KEY = '4f3f6e2b1360980aec87f727b4e6d7fb70a6169f'
VC_API_KEY = '7KJHSXG9NPA2TPE8VP52NTSPB'
# NOAA_API_KEY = 'CJGzZBJwzthMqlQrKeMtLSFKfPsSEAwV'
# AIRNOW_API_KEY = '58262ECA-A85F-4DAE-AED4-5C910AE35DDF'

cities = ['los_angeles', 'boston', 'san_francisco', 'shanghai',
           'seoul', 'paris', 'london']

# Add New Data to Feature Store

In [4]:

def get_daily_aqi(zip_code, date, distance=25):
    base_url = "https://www.airnowapi.org/aq/observation/zipCode/historical/"
    
    params = {
        'format': 'application/json',
        'zipCode': zip_code,
        'date': date,
        'distance': distance,
        'API_KEY': AIRNOW_API_KEY
    }
    # response = requests.get(base_url, params=params)
    # pm25_data = next((item for item in response.json() if item.get('ParameterName') == 'PM2.5'), None)
    # if pm25_data:
    #     return {
    #         'date': date,
    #         'ReportingArea': pm25_data.get('ReportingArea'),
    #         'StateCode': pm25_data.get('StateCode'),
    #         'Latitude': pm25_data.get('Latitude'),
    #         'Longitude': pm25_data.get('Longitude'),
    #         'pm25_aqi': pm25_data.get('AQI'),
    #         'category': pm25_data.get('Category', {}).get('Number'),
    #         'category_name': pm25_data.get('Category', {}).get('Name')
    #     }
    # return pm25_data#response.json()
    try:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            # Filter for PM2.5 data only
            pm25_data = next((item for item in data if item.get('ParameterName') == 'PM2.5'), None)
            if pm25_data:
                return {
                    'date': date,
                    'ReportingArea': pm25_data.get('ReportingArea'),
                    'StateCode': pm25_data.get('StateCode'),
                    'Latitude': pm25_data.get('Latitude'),
                    'Longitude': pm25_data.get('Longitude'),
                    'pm25_aqi': pm25_data.get('AQI'),
                    'category': pm25_data.get('Category', {}).get('Number'),
                    'category_name': pm25_data.get('Category', {}).get('Name')
                }
        return None
    except Exception as e:
        print(f"Error for {zip_code} on {date}: {str(e)}")
        return None


get_daily_aqi('10001', '2025-03-22T00-0000')

{'date': '2025-03-22T00-0000',
 'ReportingArea': 'New York City Region',
 'StateCode': 'NY',
 'Latitude': 40.8419,
 'Longitude': -73.8359,
 'pm25_aqi': 39,
 'category': 1,
 'category_name': 'Good'}

In [5]:
def get_air_json(city_name, AIR_QUALITY_API_KEY):
    return requests.get(f'https://api.waqi.info/feed/{city_name}/?token={AQOPEN_API_KEY}').json()['data']

In [12]:
def get_air_quality_data(city_name):
    # AIR_QUALITY_API_KEY = os.getenv('AIR_QUALITY_API_KEY')
    json = get_air_json(city_name, AQOPEN_API_KEY)
    print(json)
    iaqi = json['iaqi']
    forecast = json['forecast']['daily']
    
    params = ['pm10', 'pm25', 'no2', 'so2', 'co', 'o3']
    for param in params:
        if param not in iaqi: # if the parameter is not present in the data, add a column with NaN values
            iaqi[param] = {"v": np.nan}
    return [
        city_name,
        json['aqi'],                 # AQI
        json['time']['s'][:10],      # Date
        iaqi['pm25']['v'],
        iaqi['pm10']['v'],
        iaqi['no2']['v'],
        iaqi['so2']['v'],
        iaqi['co']['v'],
        iaqi['o3']['v']
    ]

In [13]:
get_air_quality_data('los angeles')#, AQOPEN_API_KEY)

{'aqi': 61, 'idx': 243, 'attributions': [{'url': 'http://www.aqmd.gov/', 'name': 'South Coast Air Quality Management District (AQMD)', 'logo': 'USA-SouthCoastAQMD.png'}, {'url': 'http://www.airnow.gov/', 'name': 'Air Now - US EPA'}, {'url': 'http://www.arb.ca.gov/', 'name': 'CARB - California Air Resources Board', 'logo': 'USA-CAARB.png'}, {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}], 'city': {'geo': [34.06653, -118.22676], 'name': 'Los Angeles-North Main Street', 'url': 'https://aqicn.org/city/losangeles/los-angeles-north-main-street', 'location': ''}, 'dominentpol': 'pm25', 'iaqi': {'co': {'v': 7.8}, 'h': {'v': 68.2}, 'no2': {'v': 19.9}, 'o3': {'v': 44.8}, 'p': {'v': 1016.7}, 'pm10': {'v': 34}, 'pm25': {'v': 61}, 'so2': {'v': 0.5}, 't': {'v': 13}, 'w': {'v': 0.4}, 'wg': {'v': 0.7}}, 'time': {'s': '2025-04-09 12:00:00', 'tz': '-07:00', 'v': 1744200000, 'iso': '2025-04-09T12:00:00-07:00'}, 'forecast': {'daily': {'uvi': [{'avg': 1, 'day': '2025-04-07', 'max'

['los angeles', 61, '2025-04-09', 61, 34, 19.9, 0.5, 7.8, 44.8]

In [ ]:
from datetime import datetime


def get_air_quality_df(data):
    col_names = [
        'city',
        'aqi',
        'date',
        'pm25',
        'pm10',
        'no2',
        'so2',
        'co',
        'o3'
    ]

    new_data = pd.DataFrame(
        data,
        columns=col_names
    )
    new_data.date = new_data.date.apply(timestamp_2_time)

    return new_data

def timestamp_2_time(x):
    dt_obj = datetime.strptime(str(x), '%Y-%m-%d')
    dt_obj = dt_obj.timestamp() * 1000
    return int(dt_obj)

In [ ]:
df = get_air_quality_df([get_air_quality_data(city) for city in cities])

In [ ]:
df

,city,aqi,date,pm25,pm10,no2,so2,co,o3
0,Brooklyn,53,1741924800000,53,NaN,NaN,NaN,NaN,NaN
1,Honolulu,37,1741924800000,37,24.0,NaN,0.1,0.9,NaN
2,Boston,31,1741924800000,31,19.0,2.8,0.3,3.4,28.1
3,Nashville,51,1741924800000,51,35.0,NaN,NaN,NaN,NaN
4,Seattle,13,1741924800000,13,NaN,12.9,NaN,NaN,NaN
5,Albuquerque,84,1741924800000,26,84.0,4.7,NaN,1.2,32.8
6,Denver,17,1741924800000,13,10.0,4.6,0.6,3.2,16.7
7,Miami,45,1741924800000,45,31.0,NaN,0.3,2.8,13.0
8,Austin,65,1741924800000,65,NaN,2.2,1.3,NaN,24.1
9,Atlanta,36,1741924800000,36,NaN,NaN,1.0,NaN,28.9


In [ ]:
# def get_air_json(city_name, AIR_QUALITY_API_KEY):
#     return requests.get(f'https://api.waqi.info/feed/{city_name}/?token={AIR_QUALITY_API_KEY}').json()['data']

In [14]:
def fetch_air_quality_data(city_name='los angeles'):
    json = requests.get(f'https://api.waqi.info/feed/{city_name}/?token={AQOPEN_API_KEY}').json()['data']
    iaqi = json['iaqi']
    forecast = json['forecast']['daily']
    
    params = ['pm10', 'pm25', 'no2', 'so2', 'co', 'o3']
    for param in params:
        if param not in iaqi: # if the parameter is not present in the data, add a column with NaN values
            iaqi[param] = {"v": np.nan}
    return pd.DataFrame(data={
        'city': [city_name],
        # json['aqi'],                 
        'date': [json['time']['s'][:10]],      # Date
        'pm25' : [iaqi['pm25']['v']],
       'pm10': [iaqi['pm10']['v']],
       'no2': [iaqi['no2']['v']],
        'so2': [iaqi['so2']['v']],
       'co': [iaqi['co']['v']],
       'o3': [iaqi['o3']['v']]
    })

In [15]:
air_quality_data = fetch_air_quality_data()

In [11]:
response = requests.get(f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/los%20angeles/today?unitGroup=us&include=days&key={VC_API_KEY}&contentType=json').json()
weather_data = pd.DataFrame(response['days'])
weather_data

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,sunrise,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source
0,2025-04-10,1744268400,80.0,56.7,66.6,80.0,56.7,66.6,50.7,59.6,...,06:27:49,1744291669,19:21:05,1744338065,0.42,Clear,Clear conditions throughout the day.,clear-day,"[KHHR, KBUR, KSMO, SE428, KLAX, F1624, CQ070, ...",comb


In [12]:
weather_data.columns


Index(['datetime', 'datetimeEpoch', 'tempmax', 'tempmin', 'temp',
       'feelslikemax', 'feelslikemin', 'feelslike', 'dew', 'humidity',
       'precip', 'precipprob', 'precipcover', 'preciptype', 'snow',
       'snowdepth', 'windgust', 'windspeed', 'winddir', 'pressure',
       'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex',
       'severerisk', 'sunrise', 'sunriseEpoch', 'sunset', 'sunsetEpoch',
       'moonphase', 'conditions', 'description', 'icon', 'stations', 'source'],
      dtype='object')

In [8]:
temp = pd.read_csv('data/processed_data/merged_la_data.csv')
temp.columns

Index(['date', 'tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover',
       'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation',
       'solarenergy', 'uvindex', 'moonphase', 'conditions', 'pm25'],
      dtype='object')

In [16]:
features = pd.concat([weather_data, air_quality_data], axis=1)
features

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,stations,source,city,date,pm25,pm10,no2,so2,co,o3
0,2025-04-10,1744268400,80.0,56.7,66.6,80.0,56.7,66.6,50.7,59.6,...,"[KHHR, KBUR, KSMO, SE428, KLAX, F1624, CQ070, ...",comb,los angeles,2025-04-10,62,32,16,0.9,4.5,40
